# **Lab 2: Matrix factorization**
**Helmer Nylén**

# **Abstract**

In this report we implement four algorithms and test these against `numpy` for accuracy. All tests pass and we conclude that the implementation is likely to be correct.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Helmer Nylén (helmern@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import scipy, scipy.sparse
from random import randint

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

In this report we implement four matrix algorithms: the sparse matrix-vector product, Gram-Schmidt QR factorization, a direct solver of $Ax = b$ and the QR eigenvalue algorithm. These were implemented using structures already present in `numpy` and `scipy`.

<!--Give a short description of the problem investigated in the report, and provide some background information so that the reader can understand the context. 

Briefly describe what method you have chosen to solve the problem, and justify why you selected that method. 

Here you can express mathematics through Latex syntax, and use hyperlinks for references.

[Hyperlink to DD2363 course website.](https://kth.instructure.com/courses/7500)

$
{\displaystyle \frac{\partial u}{\partial t}} + u\cdot \nabla u +\nabla p = f, \quad \nabla \cdot u=0$
-->


# **Methods**

Assuming `mat` is a matrix in CSR (or CRS) format and `vec` is a 1D numpy vector, this function caluclates the matrix-vector product using the method described in algorithm 5.9, chapter 5.7 of the lecture notes.

Since the csr representation in the specification of the assignment does not give information about the order of the second dimension of the matrix we instead infer the minimum order from the largest value of `col_idx`. We allow multiplications with any $x\in\mathbb{R}^{n}$ such that $n$ is greater than or equal to the inferred minimum order.

In [0]:
# Algorithm 5.9, chapter 5.7
def csr_vector_product(mat, vec):
  val, col_idx, row_ptr = mat.data, mat.indices, mat.indptr
  assert len(vec.shape) == 1 and (len(col_idx) == 0 or vec.shape[0] > max(col_idx))
  m = len(row_ptr) - 1
  res = np.zeros(m)
  for i in range(m):
    res[i] = val[row_ptr[i] : row_ptr[i+1]]\
              .dot(vec[col_idx[row_ptr[i] : row_ptr[i+1]]])
  return res  

We verify the implementation by comparing it to `numpy`s matrix-vector product.

In [0]:
def test_csr():
  n = randint(1, 10)
  m = randint(1, 10)
  mat = scipy.sparse.random(m, n, density = 0.15, format="csr")
  vec = np.random.rand(n) * 2 - 1
  assert np.allclose(mat.toarray() @ vec, csr_vector_product(mat, vec))

We assume that `mat` is a nonsingular square matrix and compute the QR factorization using a mixture of algorithm 5.3, chapter 5.3 in the lecture notes and the Matlab algorithm described in Wikipedia.

In [0]:
# Adapted from Algorithm 5.3, chapter 5.3 and https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process#Algorithm
def QR(mat):
  assert len(mat.shape) == 2 and mat.shape[0] == mat.shape[1]
  n = mat.shape[0]
  Q = np.zeros((n, n))
  R = np.zeros((n, n))
  # assume mat nonsingular
  for i in range(n):
    Q[:, i] = mat[:, i]
    for j in range(i):
      R[j, i] = Q[:, j].dot(Q[:, i]) / Q[:, j].dot(Q[:, j])
      Q[:, i] = Q[:, i] - Q[:, j] * R[j, i]
    R[i, i] = np.sqrt(Q[:, i].dot(Q[:, i]))
    Q[:, i] /= R[i, i]
  return Q, R

We verify the implementation by asserting that $R$ is upper triangular, that $Q^T = Q^{-1}$ and that $A = QR$.

In [0]:
def test_QR():
  n = randint(1, 10)
  A = np.random.rand(n, n) * 2 - 1
  while np.isclose(0, np.linalg.det(A)):
    A = np.random.rand(n, n) * 2 - 1
  Q, R = QR(A)

  # R upper triangular
  for i in range(n):
    for j in range(n):
      if i > j:
        assert R[i, j] == 0
        
  assert np.isclose(0, np.linalg.norm(Q.transpose() @ Q - np.eye(n)))
  assert np.isclose(0, np.linalg.norm(Q @ R - A))

We need two subroutines for this implementation: the QR factorization above and the backward substitution described in algorithm 5.2, chapter 5.2 of the lecture notes.

To solve $Ax = b$ we then simply substitute $A = QR$ and get $$QRx = b.$$ Using $Q^T = Q^{-1}$ we get $$Rx = Q^T b$$ and can provide this to our backward substitution to finally have $$x = R^{-1} Q^T b.$$

In [0]:
# Algorithm 5.2, chapter 5.2
def backward_substitution(U, b):
  assert U.shape[0] == U.shape[1] == b.shape[0]
  # assume U upper triangular
  n = b.shape[0]
  x = np.zeros(n)
  x[-1] = b[-1] / U[-1, -1]
  for i in range(n-2, -1, -1):
    x[i] = (b[i] - U[i, i+1:n].dot(x[i+1:n])) / U[i, i]
  return x

def solve(A, b):
  Q, R = QR(A)
  return backward_substitution(R, Q.transpose() @ b)


To verify this implementation we check that, for our computed $x$, $||Ax - b|| \approx 0$ and for a constructed $b = Ay$, solving $Ax = b$ yields $||x-y|| \approx 0$.

In [0]:
def test_solve():
  n = randint(1, 10)
  A = np.random.rand(n, n) * 2 - 1
  while np.isclose(0, np.linalg.det(A)):
    A = np.random.rand(n, n) * 2 - 1
  b = np.random.rand(n) * 2 - 1

  x = solve(A, b)
  assert np.isclose(0, np.linalg.norm(A @ x - b))

  y = np.random.rand(n) * 2 - 1
  x = solve(A, A @ y)
  if not np.isclose(0, np.linalg.norm(x - y)):
    print("[Solver] x and y slightly differ: ")
    print("x:", x)
    print("y:", y)
  assert np.isclose(0, np.linalg.norm(x - y), atol = 1e-4)

We implement the QR eigenvalue algorithm using the method described in chapter 6.6 of the lecture notes (mainly algorithm 6.1). We repeat the Schur factorization approximation until the difference in $A$ becomes small enough.

In [0]:
# Algorithm 6.1, chapter 6.6
def QR_eigenvalue_alg(mat, eps = 1e-10):
  assert mat.shape[0] == mat.shape[1]
  n = mat.shape[0]
  # assume mat real and assert that it is symmetric
  for i in range(n):
    for j in range(i):
      assert mat[i, j] == mat[j, i]

  U = np.eye(n)
  matlast = np.zeros((n, n))
  while np.linalg.norm(mat - matlast) > eps:
    matlast = mat
    Q, R = QR(mat)
    mat = R @ Q
    U = U @ Q
  return np.diag(mat), U

To verify our implementation we calculate the eigenvalues and -vectors of a random symmetric matrix $A$, and assert that $\det(A - \lambda_i I) \approx 0$ for all eigenvalues $\lambda_i$ and that $||Av_i - \lambda_i v_i|| \approx 0$ for all associated eigenvectors $v_i$.

Since these computations are more complex than those in the other implementations we need a bigger tolerance to achieve a reasonable execution time.

In [0]:
def test_QR_eig_alg():
  n = randint(1, 10)
  A = np.random.rand(n, n) * 2 - 1
  # make A symmetric
  for i in range(n):
    for j in range(i):
      A[i, j] = A[j, i]
  
  vals, vecs = QR_eigenvalue_alg(A)
  for i in range(n):
    assert np.isclose(0, np.linalg.det(A - np.eye(n) * vals[i]), atol = 1e-5)
    assert np.isclose(0, np.linalg.norm(A @ vecs[:, i] - vals[i] * vecs[:, i]), atol = 1e-5)

# **Results**

In [12]:
for i in range(500):
  print(i, end = " ")
  test_csr()
  test_QR()
  test_solve()
  test_QR_eig_alg()
print()
print("All tests passed")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

All of our constructed tests passed when using random float data for testing. We note, however, that numeric inaccuracy in the solver sometimes causes `numpy`'s `isclose` to consider the solution incorrect. Increasing the tolerance to only consider the first four decimals causes the test to pass.

# **Discussion**

Since the tests passed one of two things must be true: either the implementation is correct or the tests are not accurate. We assume that the `numpy` methods are correct and hope that our application of these methods are as well, suggesting that the implementation is satisfactory.